In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

*Barchart extraction method*

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

month_map = {
    "F": "Jan", "G": "Feb", "H": "Mar", "J": "Apr", "K": "May", "M": "Jun",
    "N": "Jul", "Q": "Aug", "U": "Sep", "V": "Oct", "X": "Nov", "Z": "Dec"
}

# invert the dictionary - so that it identifies the correct month
month_name_to_code = {v: k for k, v in month_map.items()}

def parse_barchart_symbols(base_code: str, start_year=None, end_year=None, start_month=None):
    today = datetime.today()

    # Set start year and month dynamically
    if start_year is None:
        start_year = today.year
    if start_month is None:
        current_month_abbr = today.strftime("%b")  # e.g., "Aug"
        start_month = month_name_to_code[current_month_abbr]

    # Calculate end year based on 24 months ahead
    if end_year is None:
        end_date = today + relativedelta(months=+24)
        end_year = end_date.year

    month_codes = list(month_map.keys())
    years = list(range(start_year % 100, end_year % 100 + 1))

    symbols = []
    for y in years:
        for m in month_codes:
            # Skip earlier months in the start year
            if y == start_year % 100 and month_codes.index(m) < month_codes.index(start_month):
                continue
            symbols.append(f"{base_code}{m}{y}")
    return symbols

def fetch_barchart_prices(symbols, base_url="https://www.barchart.com/futures/quotes/{}/overview"):
    headers = {"User-Agent": "Mozilla/5.0"}
    records = []
    for symbol in symbols:
        url = base_url.format(symbol)
        try:
            resp = requests.get(url, headers=headers)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, "html.parser")
            div = soup.find("div", class_="symbol-header-info")
            if not div:
                continue
            ng_init = div.get("data-ng-init")
            match = re.search(r'init\((\{.*?\})\)', ng_init)
            if not match:
                continue
            data = json.loads(match.group(1).replace(r'\/', '/'))

            raw_price = data.get("lastPrice")
            if raw_price in [None, "N/A", "-"]:
                continue
            clean_price = re.sub(r"[^\d.]+$", "", str(raw_price))

            records.append({
                "symbol": symbol,  # FIX: use passed symbol not unreliable data.get()
                "last_price": float(clean_price) if clean_price else None
            })
        except Exception:
            continue
    return pd.DataFrame(records)

def clean_and_format_df(df_raw, label):
    df = df_raw.copy()

    # Determine base code length: 2 for RB, 3 for others
    # Assume all rows in df_raw have the same base length
    sample_symbol = df["symbol"].iloc[0]
    base_code_len = 2 if sample_symbol[:2] == "RB" else 3

    # Only keep symbols that are long enough to contain month code and year
    df = df[df["symbol"].str.len() >= base_code_len + 3]

    # Extract month code dynamically
    df["month_code"] = df["symbol"].str[base_code_len]
    df["month"] = df["month_code"].map(month_map)

    # Drop rows with unrecognized month codes (e.g., bad cash contracts)
    df = df[df["month"].notna()]

    # Extract year from last two characters
    df["year"] = 2000 + df["symbol"].str[-2:].astype(int)

    # Format output
    df = df[["month", "year", "last_price"]].rename(columns={"last_price": label})
    df["order"] = pd.to_datetime(df["year"].astype(str) + "-" + df["month"], format="%Y-%b")
    return df.sort_values("order").drop(columns="order").reset_index(drop=True)

In [3]:
# 0.5 Sing flat price curve (IIH)
symbols_05 = parse_barchart_symbols("IIH")
df_05_raw = fetch_barchart_prices(symbols_05)
df_05 = clean_and_format_df(df_05_raw, "0.5-Sing flat ($/kt)")

In [4]:
# 380cst sing flat price curve (JSE)
symbols_380 = parse_barchart_symbols("JSE")
df_380_raw = fetch_barchart_prices(symbols_380)
df_380 = clean_and_format_df(df_380_raw, "380 Sing flat ($/kt)")

In [5]:
# 0.5% Euro Barges (IID)
symbols_iid = parse_barchart_symbols("IID")
df_iid_raw = fetch_barchart_prices(symbols_iid)
df_05_barge = clean_and_format_df(df_iid_raw, "0.5-Barges flat ($/kt)")

In [6]:
# 3.5% ARA Barges (JUV)
symbols_juv = parse_barchart_symbols("JUV")
df_juv_raw = fetch_barchart_prices(symbols_juv)
df_35_barge = clean_and_format_df(df_juv_raw, "3.5-Barges flat ($/kt)")

*CME extraction data*

In [7]:
# can be used for sing kero as well 
def extract_cme_flat_curve(url: str, label: str, cutoff_year: int = 2027) -> pd.DataFrame:
    """Scrapes and formats CME flat price data from a product URL."""
    
    driver = webdriver.Chrome()
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    try:
        load_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load All')]")))
        driver.execute_script("arguments[0].click();", load_button)
        time.sleep(3)
    except Exception as e:
        print(f"[{label}] Load All failed or not present:", e)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    rows = soup.find_all("tr", class_="table-row-animate")
    data = []
    for row in rows:
        tds = row.find_all("td")
        if len(tds) >= 6:
            raw_month = tds[0].text.strip()
            price_div = tds[5].find("div", class_="table-cell")
            price = price_div.text.strip() if price_div else None
            data.append({"month_raw": raw_month, "prior_settle": price})

    df = pd.DataFrame(data)

    # Parse month/year from raw text
    def extract_month_year(cell):
        # mathcing the month code to the naming convention in the sheet
        match = re.match(r"^(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)\s+(\d{4})", cell.strip(), re.IGNORECASE)
        if match:
            month = match.group(1).capitalize()
            year = int(match.group(2))
            return pd.Series([month, year])
        return pd.Series([None, None])

    df[["month", "year"]] = df["month_raw"].apply(extract_month_year)
    df[label] = pd.to_numeric(df["prior_settle"], errors="coerce")

    # Filter and final clean
    df = df[df["year"].notnull() & (df["year"] <= cutoff_year)]
    df = df[["month", "year", label]].reset_index(drop=True)
    return df

In [8]:
# 180 cst Singapore Fuel Oil
url_180 = "https://www.cmegroup.com/markets/energy/refined-products/singapore-fuel-oil-180cst-calendar-swap-futures.html"
df_180 = extract_cme_flat_curve(url_180, label="180 sing flat ($/kt)")

In [9]:
# List all your DataFrames
dfs = [df_05, df_180, df_380, df_05_barge, df_35_barge]

# Merge them one by one on 'month' and 'year'
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=["month", "year"], how="outer")

# Optional: sort for neatness
merged_df = merged_df.sort_values(by=["year", "month"]).reset_index(drop=True)
merged_df

# Step 1: Create a datetime column from month and year
merged_df["date"] = pd.to_datetime(merged_df["month"] + " " + merged_df["year"].astype(str), format="%b %Y")

# Step 2: Sort by the datetime column
merged_df = merged_df.sort_values("date").drop(columns="date").reset_index(drop=True)

#merged_df

**Implied brent swap curve**

In [10]:
# 3.5 bdge crk
symbols_jfo = parse_barchart_symbols("JFO")
df_jfo_raw = fetch_barchart_prices(symbols_jfo)
df_35_bdg_crack = clean_and_format_df(df_jfo_raw, "3.5 Bdge Crk ($/bbl)")
#df_35_bdg_crack

In [11]:
merged_df = pd.merge(merged_df, df_35_bdg_crack, on=["month", "year"], how="left")

In [12]:
# Convert 3.5% Barge flat price from $/kt to $/bbl
merged_df["3.5 bdge flat ($/bbl)"] = merged_df["3.5-Barges flat ($/kt)"] / 6.35

# Calculate Brent swap by subtracting crack from flat price in $/bbl 
merged_df["Implied Brent swap ($/bbl)"] = merged_df["3.5 bdge flat ($/bbl)"] - merged_df["3.5 Bdge Crk ($/bbl)"]

# implied curve derived by the barge crk curve

In [13]:
#merged_df

**Mid Disti**

In [14]:
# Singapore Jet Kerosene
url_jet = "https://www.cmegroup.com/markets/energy/refined-products/singapore-jet-kerosene-swap-futures.html"
df_sing_kero = extract_cme_flat_curve(url_jet, label="Sing Kero flat ($/bbl)")

In [15]:
# Sing 10ppm 
symbols_jsg = parse_barchart_symbols("JSG")
df_jsg_raw = fetch_barchart_prices(symbols_jsg)
df_10ppm = clean_and_format_df(df_jsg_raw, "10ppm flat ($/bbl)")

In [16]:
# LSGO flat price
symbols_lf = parse_barchart_symbols("LF")
df_lf_raw = fetch_barchart_prices(symbols_lf)

def clean_lf_format(df_raw, label):
    df = df_raw.copy()
    df["month_code"] = df["symbol"].str[2]  # Month at index 2 for 'LFU25'
    df["month"] = df["month_code"].map(month_map)
    df["year"] = 2000 + df["symbol"].str[-2:].astype(int)
    df = df[["month", "year", "last_price"]].rename(columns={"last_price": label})
    df["order"] = pd.to_datetime(df["year"].astype(str) + "-" + df["month"], format="%Y-%b", errors='coerce')
    return df.sort_values("order").drop(columns="order").reset_index(drop=True)

df_lsgo_flat = clean_lf_format(df_lf_raw, "LSGO flat ($/kt)")

In [17]:
#df_lsgo_flat = df_lsgo_flat.drop(columns=["month"])
#df_lsgo_flat

In [18]:
merged_df = merged_df.merge(df_sing_kero, on=["month", "year"], how="left")
merged_df = merged_df.merge(df_10ppm, on=["month", "year"], how="left")
#merged_df = merged_df.merge(df_lsgo_flat, on=["month", "year"], how="left")

In [19]:
merged_df = merged_df.merge(df_lsgo_flat, on=["month", "year"], how="left")
#merged_df

**Gasoline additions**

In [20]:
# M92 scrape
symbols_m92 = parse_barchart_symbols("J1N")
df_m92_raw = fetch_barchart_prices(symbols_m92)
df_m92 = clean_and_format_df(df_m92_raw, "M92 flat ($/bbl)")

In [21]:
# EBOB Scrape 
symbols_ebob = parse_barchart_symbols("J7H")
df_ebob_raw = fetch_barchart_prices(symbols_ebob)
df_ebob = clean_and_format_df(df_ebob_raw, "EBOB flat ($/kt)")

In [22]:
# RBOB Scrape
# Get the month after the current month (this month has already settled)
next_month_abbr = (datetime.today() + relativedelta(months=+1)).strftime("%b")
next_month_code = month_name_to_code[next_month_abbr]

symbols_rbob = parse_barchart_symbols("RB", start_month=next_month_code)
df_rbob_raw = fetch_barchart_prices(symbols_rbob)
df_rbob = clean_and_format_df(df_rbob_raw, "RBOB flat ($/gal)")

In [23]:
df_rbob # need to ensure that the correct dates end up matching off
# this was not working out due to the presence of a cash contract

,month,year,RBOB flat ($/gal)
0,Sep,2025,2.1002
1,Oct,2025,1.9469
2,Nov,2025,1.8977
3,Dec,2025,1.8600
4,Jan,2026,1.8451
5,Feb,2026,1.8495
6,Mar,2026,1.8652
7,Apr,2026,2.0509
8,May,2026,2.0536
9,Jun,2026,2.0518


In [24]:
merged_df = merged_df.merge(df_m92, on=["month", "year"], how="left")
merged_df = merged_df.merge(df_ebob, on=["month", "year"], how="left")
merged_df = merged_df.merge(df_rbob, on=["month", "year"], how = "left")
merged_df

,month,year,0.5-Sing flat ($/kt),180 sing flat ($/kt),380 Sing flat ($/kt),0.5-Barges flat ($/kt),3.5-Barges flat ($/kt),3.5 Bdge Crk ($/bbl),3.5 bdge flat ($/bbl),Implied Brent swap ($/bbl),Sing Kero flat ($/bbl),10ppm flat ($/bbl),LSGO flat ($/kt),M92 flat ($/bbl),EBOB flat ($/kt),RBOB flat ($/gal)
0,Aug,2025,490.868,460.375,406.334,465.046,414.446,-2.491,65.267087,67.758087,97.862,86.927,668.50,76.806,690.644,NaN
1,Sep,2025,487.878,455.875,407.942,458.044,407.025,-2.850,64.098425,66.948425,97.432,85.113,662.75,75.709,676.472,2.1002
2,Oct,2025,484.386,451.125,402.950,452.211,397.775,-3.835,62.641732,66.476732,97.082,84.259,660.00,74.524,637.595,1.9469
3,Nov,2025,480.003,447.750,397.067,446.294,390.275,-4.720,61.460630,66.180630,96.690,83.280,651.75,73.243,615.218,1.8977
4,Dec,2025,476.211,445.250,392.858,442.128,384.275,-5.484,60.515748,65.999748,96.260,82.474,642.50,72.327,600.524,1.8600
5,Jan,2026,473.886,444.000,391.117,NaN,380.900,-5.912,59.984252,65.896252,95.934,82.183,639.50,71.869,599.603,1.8451
6,Feb,2026,471.115,443.000,389.346,NaN,379.150,-6.097,59.708661,65.805661,95.737,82.101,637.25,72.000,603.280,1.8495
7,Mar,2026,468.469,442.000,387.783,NaN,378.775,-6.076,59.649606,65.725606,95.421,82.011,633.25,72.204,608.435,1.8652
8,Apr,2026,465.878,441.000,386.275,432.878,378.525,-6.036,59.610236,65.646236,94.952,81.870,629.50,72.113,644.278,2.0509
9,May,2026,463.836,440.000,384.400,430.586,377.400,-6.123,59.433071,65.556071,94.571,81.801,627.75,71.886,645.492,2.0536


**Naphtha additions**

In [25]:
# Naphtha Scrpae
symbols_MOPJ = parse_barchart_symbols("JJA")
df_MOPJ_raw = fetch_barchart_prices(symbols_MOPJ)
df_MOPJ = clean_and_format_df(df_MOPJ_raw, "MOPJ flat ($/kt)")

In [26]:
merged_df = merged_df.merge(df_MOPJ, on=["month", "year"], how="left")

In [27]:
# Ensure sorting
merged_df["order"] = pd.to_datetime(merged_df["year"].astype(int).astype(str) + "-" + merged_df["month"], format="%Y-%b")
merged_df = merged_df.sort_values("order").reset_index(drop=True)

# Compute Jul–Aug style time spreads
col = "0.5-Sing flat ($/kt)"
ts_col = "0.5-Sing TS ($/kt)"

merged_df[ts_col] = merged_df.apply(
    lambda x: x[col] - merged_df[col].shift(-1).loc[x.name]
    if pd.notnull(x[col]) and pd.notnull(merged_df[col].shift(-1).loc[x.name])
    else np.nan,
    axis=1
)

# Drop helper column
merged_df = merged_df.drop(columns="order")

In [28]:
# Ensure merged_df is sorted by date
merged_df["order"] = pd.to_datetime(
    merged_df["year"].astype(int).astype(str) + "-" + merged_df["month"],
    format="%Y-%b"
)
merged_df = merged_df.sort_values("order").reset_index(drop=True)

# List of flat price columns to calculate time spreads for
flat_columns = [
    "0.5-Sing flat ($/kt)", # remove code above for the 0.5-Sing flat tmrw
    "180 sing flat ($/kt)",
    "380 Sing flat ($/kt)",
    "0.5-Barges flat ($/kt)",
    "3.5-Barges flat ($/kt)",
    "Sing Kero flat ($/bbl)",
    "10ppm flat ($/bbl)",
    "LSGO flat ($/kt)",
    "M92 flat ($/bbl)",
    "EBOB flat ($/kt)",
    "RBOB flat ($/gal)",
    "MOPJ flat ($/kt)"
]

# Compute TS for each and append new column
for col in flat_columns:
    ts_col = col.replace("flat", "TS")
    merged_df[ts_col] = merged_df[col] - merged_df[col].shift(-1)

# Drop sorting helper
merged_df = merged_df.drop(columns="order")
merged_df

,month,year,0.5-Sing flat ($/kt),180 sing flat ($/kt),380 Sing flat ($/kt),0.5-Barges flat ($/kt),3.5-Barges flat ($/kt),3.5 Bdge Crk ($/bbl),3.5 bdge flat ($/bbl),Implied Brent swap ($/bbl),...,380 Sing TS ($/kt),0.5-Barges TS ($/kt),3.5-Barges TS ($/kt),Sing Kero TS ($/bbl),10ppm TS ($/bbl),LSGO TS ($/kt),M92 TS ($/bbl),EBOB TS ($/kt),RBOB TS ($/gal),MOPJ TS ($/kt)
0,Aug,2025,490.868,460.375,406.334,465.046,414.446,-2.491,65.267087,67.758087,...,-1.608,7.002,7.421,0.430,1.814,5.75,1.097,14.172,NaN,5.013
1,Sep,2025,487.878,455.875,407.942,458.044,407.025,-2.850,64.098425,66.948425,...,4.992,5.833,9.250,0.350,0.854,2.75,1.185,38.877,0.1533,3.259
2,Oct,2025,484.386,451.125,402.950,452.211,397.775,-3.835,62.641732,66.476732,...,5.883,5.917,7.500,0.392,0.979,8.25,1.281,22.377,0.0492,3.208
3,Nov,2025,480.003,447.750,397.067,446.294,390.275,-4.720,61.460630,66.180630,...,4.209,4.166,6.000,0.430,0.806,9.25,0.916,14.694,0.0377,3.036
4,Dec,2025,476.211,445.250,392.858,442.128,384.275,-5.484,60.515748,65.999748,...,1.741,NaN,3.375,0.326,0.291,3.00,0.458,0.921,0.0149,2.596
5,Jan,2026,473.886,444.000,391.117,NaN,380.900,-5.912,59.984252,65.896252,...,1.771,NaN,1.750,0.197,0.082,2.25,-0.131,-3.677,-0.0044,2.566
6,Feb,2026,471.115,443.000,389.346,NaN,379.150,-6.097,59.708661,65.805661,...,1.563,NaN,0.375,0.316,0.090,4.00,-0.204,-5.155,-0.0157,3.401
7,Mar,2026,468.469,442.000,387.783,NaN,378.775,-6.076,59.649606,65.725606,...,1.508,NaN,0.250,0.469,0.141,3.75,0.091,-35.843,-0.1857,4.403
8,Apr,2026,465.878,441.000,386.275,432.878,378.525,-6.036,59.610236,65.646236,...,1.875,2.292,1.125,0.381,0.069,1.75,0.227,-1.214,-0.0027,3.604
9,May,2026,463.836,440.000,384.400,430.586,377.400,-6.123,59.433071,65.556071,...,2.500,1.650,1.750,0.230,0.098,0.25,0.350,2.423,0.0018,3.020


*Cracks*

In [29]:
# keeping a df prior to including cracks (in case of any fuck ups)
merged_df_for_crks = merged_df.copy()

# use merged_df_for_crks in the subsequent steps

In [30]:
# Conversion factors
conversion_factors = {
    "180 sing flat ($/kt)": 6.35,
    "380 Sing flat ($/kt)": 6.35,
    "0.5-Barges flat ($/kt)": 6.35,
    "0.5-Sing flat ($/kt)": 6.35,
    "LSGO flat ($/kt)": 7.45,
    "EBOB flat ($/kt)": 8.33,
    "MOPJ flat ($/kt)": 8.9
}

# No conversion needed
no_conversion = ["Sing Kero flat ($/bbl)", "10ppm flat ($/bbl)", "M92 flat ($/bbl)"]

## Maybe create a separate list or dictionary for RBOB as it prices in $/gal 
gal_conversion_factors = {
    "RBOB flat ($/gal)": 42.0 # implement for cracks then have to do the kt conversion for the "Arb"
    ##  Remember:
    # RBBRs are priced in $/bbl, 
    # the "ARB (RBOB v EBOB)" priced in $/gal also give the $/kt & $/bbl cols as well
    # E/W Gasoline priced in $/bbl
}

for col in gal_conversion_factors:
    if col in merged_df_for_crks.columns:
        crack_col = col.replace("flat", "crk").replace("$/gal", "$/bbl").strip()
        merged_df_for_crks[crack_col] = merged_df_for_crks.apply(
            lambda row: (row[col] * gal_conversion_factors[col]) - row["Implied Brent swap ($/bbl)"]
            if pd.notnull(row[col]) and pd.notnull(row["Implied Brent swap ($/bbl)"]) else np.nan,
            axis=1
        )

# Calculate crack spreads
for col in conversion_factors:
    if col in merged_df_for_crks.columns:
        crack_col = col.replace("flat", "crk").replace("(kt)", "($/bbl)").strip()
        merged_df_for_crks[crack_col] = merged_df_for_crks.apply(
            lambda row: (row[col] / conversion_factors[col]) - row["Implied Brent swap ($/bbl)"]
            if pd.notnull(row[col]) and pd.notnull(row["Implied Brent swap ($/bbl)"]) else np.nan,
            axis=1
        )

for col in no_conversion:
    if col in merged_df_for_crks.columns:
        crack_col = col.replace("flat", "crk").strip()
        merged_df_for_crks[crack_col] = merged_df_for_crks.apply(
            lambda row: row[col] - row["Implied Brent swap ($/bbl)"]
            if pd.notnull(row[col]) and pd.notnull(row["Implied Brent swap ($/bbl)"]) else np.nan,
            axis=1
            )

# Final output
merged_df_for_blends = merged_df_for_crks.copy() # keep merged_df_for_crks as a reference in case of any fuck ups

In [31]:
# Add Viz spread ($/kt)
merged_df_for_blends["Vis spread (180 v 380) ($/kt)"] = merged_df_for_blends.apply(
    lambda row: row["180 sing flat ($/kt)"] - row["380 Sing flat ($/kt)"]
    if pd.notnull(row["180 sing flat ($/kt)"]) and pd.notnull(row["380 Sing flat ($/kt)"]) else np.nan,
    axis=1
)

# Add Regrade ($/bbl)
merged_df_for_blends["Regrade ($/bbl)"] = merged_df_for_blends.apply(
    lambda row: row["Sing Kero flat ($/bbl)"] - row["10ppm flat ($/bbl)"]
    if pd.notnull(row["Sing Kero flat ($/bbl)"]) and pd.notnull(row["10ppm flat ($/bbl)"]) else np.nan,
    axis=1
)

# Add each of the 92-MOPJ curves
merged_df_for_blends["M92 v MOPJ ($/bbl) 8.9-conv"] = merged_df_for_blends.apply(
    lambda row: row["M92 flat ($/bbl)"] - (row["MOPJ flat ($/kt)"]/8.9)
    if pd.notnull(row["M92 flat ($/bbl)"]) and pd.notnull(row["MOPJ flat ($/kt)"]) else np.nan,
    axis=1
)

merged_df_for_blends["M92 v MOPJ ($/bbl) 9.0-conv"] = merged_df_for_blends.apply(
    lambda row: row["M92 flat ($/bbl)"] - (row["MOPJ flat ($/kt)"]/9.0)
    if pd.notnull(row["M92 flat ($/bbl)"]) and pd.notnull(row["MOPJ flat ($/kt)"]) else np.nan,
    axis=1
)

merged_df_for_geo = merged_df_for_blends.copy() # keep copy df in case of fuckups
# merged_df_geo to add the E/W and geo arbs
merged_df_for_blends

,month,year,0.5-Sing flat ($/kt),180 sing flat ($/kt),380 Sing flat ($/kt),0.5-Barges flat ($/kt),3.5-Barges flat ($/kt),3.5 Bdge Crk ($/bbl),3.5 bdge flat ($/bbl),Implied Brent swap ($/bbl),...,LSGO crk ($/kt),EBOB crk ($/kt),MOPJ crk ($/kt),Sing Kero crk ($/bbl),10ppm crk ($/bbl),M92 crk ($/bbl),Vis spread (180 v 380) ($/kt),Regrade ($/bbl),M92 v MOPJ ($/bbl) 8.9-conv,M92 v MOPJ ($/bbl) 9.0-conv
0,Aug,2025,490.868,460.375,406.334,465.046,414.446,-2.491,65.267087,67.758087,...,21.973457,15.152358,-2.838536,30.103913,19.168913,9.047913,54.041,10.935,11.886449,12.607778
1,Sep,2025,487.878,455.875,407.942,458.044,407.025,-2.850,64.098425,66.948425,...,22.011306,14.260698,-2.592133,30.483575,18.164575,8.760575,47.933,12.319,11.352708,12.067778
2,Oct,2025,484.386,451.125,402.950,452.211,397.775,-3.835,62.641732,66.476732,...,22.113872,10.065285,-2.486620,30.605268,17.782268,8.047268,48.175,12.823,10.533888,11.244889
3,Nov,2025,480.003,447.750,397.067,446.294,390.275,-4.720,61.460630,66.180630,...,21.302592,7.675072,-2.550967,30.509370,17.099370,7.062370,50.683,13.410,9.613337,10.320333
4,Dec,2025,476.211,445.250,392.858,442.128,384.275,-5.484,60.515748,65.999748,...,20.241863,6.091969,-2.711209,30.260252,16.474252,6.327252,52.392,13.786,9.038461,9.741667
5,Jan,2026,473.886,444.000,391.117,NaN,380.900,-5.912,59.984252,65.896252,...,19.942674,6.084900,-2.899398,30.037748,16.286748,5.972748,52.883,13.751,8.872146,9.572111
6,Feb,2026,471.115,443.000,389.346,NaN,379.150,-6.097,59.708661,65.805661,...,19.731251,6.616908,-3.097122,29.931339,16.295339,6.194339,53.654,13.636,9.291461,9.988222
7,Mar,2026,468.469,442.000,387.783,NaN,378.775,-6.076,59.649606,65.725606,...,19.274394,7.315810,-3.399202,29.695394,16.285394,6.478394,54.217,13.410,9.877596,10.570111
8,Apr,2026,465.878,441.000,386.275,432.878,378.525,-6.036,59.610236,65.646236,...,18.850408,11.698061,-3.814551,29.305764,16.223764,6.466764,54.725,13.082,10.281315,10.968333
9,May,2026,463.836,440.000,384.400,430.586,377.400,-6.123,59.433071,65.556071,...,18.705674,11.933965,-4.129329,29.014929,16.244929,6.329929,55.600,12.770,10.459258,11.141778


In [32]:
merged_df_for_geo["0.5 E/W ($/kt)"] = merged_df_for_geo.apply(
    lambda row: row["0.5-Sing flat ($/kt)"] - row["0.5-Barges flat ($/kt)"]
    if pd.notnull(row["0.5-Sing flat ($/kt)"]) and pd.notnull(row["0.5-Barges flat ($/kt)"]) else np.nan,
    axis=1
)

merged_df_for_geo["380 E/W ($/kt)"] = merged_df_for_geo.apply(
    lambda row: row["380 Sing flat ($/kt)"] - row["3.5-Barges flat ($/kt)"]
    if pd.notnull(row["380 Sing flat ($/kt)"]) and pd.notnull(row["3.5-Barges flat ($/kt)"]) else np.nan,
    axis=1
)

merged_df_for_geo["Sing Hi-5 ($/kt)"] = merged_df_for_geo.apply(
    lambda row: row["0.5-Sing flat ($/kt)"] - row["380 Sing flat ($/kt)"]
    if pd.notnull(row["0.5-Sing flat ($/kt)"]) and pd.notnull(row["380 Sing flat ($/kt)"]) else np.nan,
    axis=1
)

## Additions we need to make for the Gasoline diffs 

# E/W Gasoline
merged_df_for_geo["E/W Gasoline ($/bbl)"] = merged_df_for_geo.apply(
   lambda row: row["M92 flat ($/bbl)"] - (row["EBOB flat ($/kt)"]/8.33)
   if pd.notnull(row["M92 flat ($/bbl)"]) and pd.notnull(row["EBOB flat ($/kt)"]) else np.nan,
   axis=1
)

# Arb on Gasoline (RBOB v EBOB) in $/gal
merged_df_for_geo["Gasoline ARB ~ rbob v ebob ($/gal)"] = merged_df_for_geo.apply(
   lambda row: row["RBOB flat ($/gal)"] - ((row["EBOB flat ($/kt)"]/8.33)/42)
   if pd.notnull(row["RBOB flat ($/gal)"]) and pd.notnull(row["EBOB flat ($/kt)"]) else np.nan,
   axis=1
)

# Arb on Gasoline (RBOB v EBOB) in $/bbl
merged_df_for_geo["Gasoline ARB ~ rbob v ebob ($/bbl)"] = merged_df_for_geo.apply(
   lambda row: (row["RBOB flat ($/gal)"]*42) - ((row["EBOB flat ($/kt)"]/8.33))
   if pd.notnull(row["RBOB flat ($/gal)"]) and pd.notnull(row["EBOB flat ($/kt)"]) else np.nan,
   axis=1
)

In [33]:
# Mid Disti E/W = (10ppm flat $/bbl * 7.45) - LSGO flat ($/kt)
#merged_df_for_geo["Mid Disti E/W ($/kt)"] = (
#    merged_df_for_geo["10ppm flat ($/bbl)"] * 7.45 - merged_df_for_geo["LSGO flat ($/kt)"]
#) # smth wrong here

merged_df_for_geo

,month,year,0.5-Sing flat ($/kt),180 sing flat ($/kt),380 Sing flat ($/kt),0.5-Barges flat ($/kt),3.5-Barges flat ($/kt),3.5 Bdge Crk ($/bbl),3.5 bdge flat ($/bbl),Implied Brent swap ($/bbl),...,Vis spread (180 v 380) ($/kt),Regrade ($/bbl),M92 v MOPJ ($/bbl) 8.9-conv,M92 v MOPJ ($/bbl) 9.0-conv,0.5 E/W ($/kt),380 E/W ($/kt),Sing Hi-5 ($/kt),E/W Gasoline ($/bbl),Gasoline ARB ~ rbob v ebob ($/gal),Gasoline ARB ~ rbob v ebob ($/bbl)
0,Aug,2025,490.868,460.375,406.334,465.046,414.446,-2.491,65.267087,67.758087,...,54.041,10.935,11.886449,12.607778,25.822,-8.112,84.534,-6.104444,NaN,NaN
1,Sep,2025,487.878,455.875,407.942,458.044,407.025,-2.850,64.098425,66.948425,...,47.933,12.319,11.352708,12.067778,29.834,0.917,79.936,-5.500124,0.166649,6.999276
2,Oct,2025,484.386,451.125,402.950,452.211,397.775,-3.835,62.641732,66.476732,...,48.175,12.823,10.533888,11.244889,32.175,5.175,81.436,-2.018017,0.124471,5.227783
3,Nov,2025,480.003,447.750,397.067,446.294,390.275,-4.720,61.460630,66.180630,...,50.683,13.410,9.613337,10.320333,33.709,6.792,82.936,-0.612702,0.139231,5.847698
4,Dec,2025,476.211,445.250,392.858,442.128,384.275,-5.484,60.515748,65.999748,...,52.392,13.786,9.038461,9.741667,34.083,8.583,83.353,0.235283,0.143531,6.028283
5,Jan,2026,473.886,444.000,391.117,NaN,380.900,-5.912,59.984252,65.896252,...,52.883,13.751,8.872146,9.572111,NaN,10.217,82.769,-0.112152,0.131263,5.513048
6,Feb,2026,471.115,443.000,389.346,NaN,379.150,-6.097,59.708661,65.805661,...,53.654,13.636,9.291461,9.988222,NaN,10.196,81.769,-0.422569,0.125153,5.256431
7,Mar,2026,468.469,442.000,387.783,NaN,378.775,-6.076,59.649606,65.725606,...,54.217,13.410,9.877596,10.570111,NaN,9.008,80.686,-0.837417,0.126119,5.296983
8,Apr,2026,465.878,441.000,386.275,432.878,378.525,-6.036,59.610236,65.646236,...,54.725,13.082,10.281315,10.968333,33.000,7.750,79.603,-5.231298,0.209369,8.793502
9,May,2026,463.836,440.000,384.400,430.586,377.400,-6.123,59.433071,65.556071,...,55.600,12.770,10.459258,11.141778,33.250,7.000,79.436,-5.604036,0.208599,8.761164


In [34]:
print(merged_df_for_geo.columns.tolist())

['month', 'year', '0.5-Sing flat ($/kt)', '180 sing flat ($/kt)', '380 Sing flat ($/kt)', '0.5-Barges flat ($/kt)', '3.5-Barges flat ($/kt)', '3.5 Bdge Crk ($/bbl)', '3.5 bdge flat ($/bbl)', 'Implied Brent swap ($/bbl)', 'Sing Kero flat ($/bbl)', '10ppm flat ($/bbl)', 'LSGO flat ($/kt)', 'M92 flat ($/bbl)', 'EBOB flat ($/kt)', 'RBOB flat ($/gal)', 'MOPJ flat ($/kt)', '0.5-Sing TS ($/kt)', '180 sing TS ($/kt)', '380 Sing TS ($/kt)', '0.5-Barges TS ($/kt)', '3.5-Barges TS ($/kt)', 'Sing Kero TS ($/bbl)', '10ppm TS ($/bbl)', 'LSGO TS ($/kt)', 'M92 TS ($/bbl)', 'EBOB TS ($/kt)', 'RBOB TS ($/gal)', 'MOPJ TS ($/kt)', 'RBOB crk ($/bbl)', '180 sing crk ($/kt)', '380 Sing crk ($/kt)', '0.5-Barges crk ($/kt)', '0.5-Sing crk ($/kt)', 'LSGO crk ($/kt)', 'EBOB crk ($/kt)', 'MOPJ crk ($/kt)', 'Sing Kero crk ($/bbl)', '10ppm crk ($/bbl)', 'M92 crk ($/bbl)', 'Vis spread (180 v 380) ($/kt)', 'Regrade ($/bbl)', 'M92 v MOPJ ($/bbl) 8.9-conv', 'M92 v MOPJ ($/bbl) 9.0-conv', '0.5 E/W ($/kt)', '380 E/W ($/k

In [35]:
# List of columns to keep (your finalized structure)
columns_to_keep = [
    'month', 'year',
    'Implied Brent swap ($/bbl)',
    '0.5-Sing flat ($/kt)',
    '0.5-Sing TS ($/kt)',
    '0.5-Sing crk ($/kt)',
    'Sing Hi-5 ($/kt)',
    '380 Sing flat ($/kt)',
    '380 Sing TS ($/kt)',
    '380 Sing crk ($/kt)',
    '0.5 E/W ($/kt)',
    '380 E/W ($/kt)',
    '0.5-Barges flat ($/kt)',
    '0.5-Barges TS ($/kt)',
    '0.5-Barges crk ($/kt)',
    '3.5-Barges flat ($/kt)',
    '3.5-Barges TS ($/kt)',
    '3.5 Bdge Crk ($/bbl)',
    '10ppm flat ($/bbl)',
    '10ppm TS ($/bbl)',
    '10ppm crk ($/bbl)',
    'LSGO flat ($/kt)',
    'LSGO TS ($/kt)',
    'LSGO crk ($/kt)',
    'M92 flat ($/bbl)',
    'M92 TS ($/bbl)',
    'M92 crk ($/bbl)',
    'E/W Gasoline ($/bbl)',
    'EBOB flat ($/kt)',
    'EBOB TS ($/kt)',
    'EBOB crk ($/kt)',
    'Gasoline ARB ~ rbob v ebob ($/gal)',
    'Gasoline ARB ~ rbob v ebob ($/bbl)',
    'RBOB flat ($/gal)',
    'RBOB TS ($/gal)',
    'RBOB crk ($/bbl)',
    "M92 v MOPJ ($/bbl) 8.9-conv", # this is the one usually referred to in the mkt 
    "M92 v MOPJ ($/bbl) 9.0-conv",
    "MOPJ flat ($/kt)",
    "MOPJ TS ($/kt)",
    "MOPJ crk ($/kt)"
]

# Drop everything else
master_df = merged_df_for_geo[columns_to_keep]

In [36]:
# Correct mislabeled crack columns from ($/kt) to ($/bbl)
master_df.rename(columns={
    '0.5-Sing crk ($/kt)': '0.5-Sing crk ($/bbl)',
    '380 Sing crk ($/kt)': '380 Sing crk ($/bbl)',
    '0.5-Barges crk ($/kt)': '0.5-Barges crk ($/bbl)',
    'LSGO crk ($/kt)': 'LSGO crk ($/bbl)',
    'EBOB crk ($/kt)': 'EBOB crk ($/bbl)',
    'MOPJ crk ($/kt)': 'MOPJ crk ($/bbl)',
    'RBOB crk ($/bbl)': 'RBOB crk (RBBR) ~ ($/bbl)'
}, inplace=True)
#master_df

C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_23580\2533682713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df.rename(columns={


In [37]:
# Define special columns for 4 decimal places
rbob_cols_4dp = [
    'RBOB flat ($/gal)',
    'RBOB TS ($/gal)',
    'Gasoline ARB ~ rbob v ebob ($/gal)'
]

# Get all other numeric columns (excluding 'month', 'year', and rbob_cols_4dp)
cols_to_format_2dp = master_df.columns.difference(['month', 'year'] + rbob_cols_4dp)

# Apply 2 decimal place rounding
master_df[cols_to_format_2dp] = master_df[cols_to_format_2dp].applymap(
    lambda x: round(x, 2) if pd.notnull(x) else x
)

# Apply 4 decimal place rounding to RBOB-related columns
master_df[rbob_cols_4dp] = master_df[rbob_cols_4dp].applymap(
    lambda x: round(x, 4) if pd.notnull(x) else x
)

C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_23580\1047480198.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  master_df[cols_to_format_2dp] = master_df[cols_to_format_2dp].applymap(
C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_23580\1047480198.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df[cols_to_format_2dp] = master_df[cols_to_format_2dp].applymap(
C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_23580\1047480198.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  master_df[rbob_cols_4dp] = master_df[rbob_cols_4dp].applymap(
C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_23580\1047480198.py:17: SettingWithCopyWarning: 
A value

Extracting/making a Eur-dollar curve to place on the second sheet for Ola

In [38]:
# fetching the info
def fetch_forward_fx_rates(url, tenor_list):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    data = []

    for tenor in tenor_list:
        td = soup.find("td", string=lambda s: s and tenor in s)
        if not td:
            print(f"{tenor} not found.")
            continue

        siblings = td.find_next_siblings("td", limit=3)
        if len(siblings) < 3:
            print(f"Not enough data for {tenor}.")
            continue

        try:
            bid = float(siblings[0].text.strip())
            ask = float(siblings[1].text.strip())
            mid = float(siblings[2].text.strip())
            data.append({
                "tenor": tenor,
                "bid": round(bid, 4),
                "ask": round(ask, 4),
                "mid": round(mid, 4)
            })
        except Exception as e:
            print(f"Error parsing {tenor}: {e}")
            continue

    return pd.DataFrame(data)

tenors = [
    "Overnight", "Tomorrow Next", "Spot Next", # short term tenors
    "One Week", "Two Week", "Three Week", # week to 3 week tenors
    "One Month", "Two Month", "Three Month", "Four Month", "Five Month", "Six Month",
    "Seven Month", "Eight Month", "Nine Month", "Ten Month", "Eleven Month",
    "One Year", "Two Year", "Three Year", "Four Year", "Five Year" # mid to back rate refs
]
tenor_eur_dol = tenors + ['Six Year', 'Seven Year', 'Ten Year']

eur_dol_url = "https://www.fxempire.com/currencies/eur-usd/forward-rates"
eurdol_fwd_data = fetch_forward_fx_rates(eur_dol_url, tenor_eur_dol)

In [39]:
# creating the tenor column - better readability
master_df["tenor"] = master_df.apply(
    lambda row: f"{row['month'][:3]}'{str(row['year'])[-2:]}", axis=1
)

# drop the original columns that we no longer need
master_df.drop(columns=["month", "year"], inplace=True)

# Move 'tenor' to the first column
cols = master_df.columns.tolist()
cols.insert(0, cols.pop(cols.index("tenor")))
master_df = master_df[cols]
master_df

C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_23580\3647837345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df["tenor"] = master_df.apply(
C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_23580\3647837345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df.drop(columns=["month", "year"], inplace=True)


,tenor,Implied Brent swap ($/bbl),0.5-Sing flat ($/kt),0.5-Sing TS ($/kt),0.5-Sing crk ($/bbl),Sing Hi-5 ($/kt),380 Sing flat ($/kt),380 Sing TS ($/kt),380 Sing crk ($/bbl),0.5 E/W ($/kt),...,Gasoline ARB ~ rbob v ebob ($/gal),Gasoline ARB ~ rbob v ebob ($/bbl),RBOB flat ($/gal),RBOB TS ($/gal),RBOB crk (RBBR) ~ ($/bbl),M92 v MOPJ ($/bbl) 8.9-conv,M92 v MOPJ ($/bbl) 9.0-conv,MOPJ flat ($/kt),MOPJ TS ($/kt),MOPJ crk ($/bbl)
0,Aug'25,67.76,490.87,2.99,9.54,84.53,406.33,-1.61,-3.77,25.82,...,NaN,NaN,NaN,NaN,NaN,11.89,12.61,577.78,5.01,-2.84
1,Sep'25,66.95,487.88,3.49,9.88,79.94,407.94,4.99,-2.71,29.83,...,0.1666,7.00,2.1002,0.1533,21.26,11.35,12.07,572.77,3.26,-2.59
2,Oct'25,66.48,484.39,4.38,9.80,81.44,402.95,5.88,-3.02,32.18,...,0.1245,5.23,1.9469,0.0492,15.29,10.53,11.24,569.51,3.21,-2.49
3,Nov'25,66.18,480.00,3.79,9.41,82.94,397.07,4.21,-3.65,33.71,...,0.1392,5.85,1.8977,0.0377,13.52,9.61,10.32,566.30,3.04,-2.55
4,Dec'25,66.00,476.21,2.32,8.99,83.35,392.86,1.74,-4.13,34.08,...,0.1435,6.03,1.8600,0.0149,12.12,9.04,9.74,563.27,2.60,-2.71
5,Jan'26,65.90,473.89,2.77,8.73,82.77,391.12,1.77,-4.30,NaN,...,0.1313,5.51,1.8451,-0.0044,11.60,8.87,9.57,560.67,2.57,-2.90
6,Feb'26,65.81,471.12,2.65,8.39,81.77,389.35,1.56,-4.49,NaN,...,0.1252,5.26,1.8495,-0.0157,11.87,9.29,9.99,558.11,3.40,-3.10
7,Mar'26,65.73,468.47,2.59,8.05,80.69,387.78,1.51,-4.66,NaN,...,0.1261,5.30,1.8652,-0.1857,12.61,9.88,10.57,554.71,4.40,-3.40
8,Apr'26,65.65,465.88,2.04,7.72,79.60,386.27,1.88,-4.82,33.00,...,0.2094,8.79,2.0509,-0.0027,20.49,10.28,10.97,550.30,3.60,-3.81
9,May'26,65.56,463.84,1.65,7.49,79.44,384.40,2.50,-5.02,33.25,...,0.2086,8.76,2.0536,0.0018,20.70,10.46,11.14,546.70,3.02,-4.13


In [40]:
import pandas as pd
from datetime import datetime

# Create a filename with today's date
today_str = datetime.today().strftime("%Y-%m-%d")
filename = f"cross_bbl_pricing_sheet_{today_str}.xlsx"

# Define RBOB-specific 4dp columns
rbob_cols_4dp = [
    'RBOB flat ($/gal)',
    'RBOB TS ($/gal)',
    'Gasoline ARB ~ rbob v ebob ($/gal)'
]

# Export the DataFrames with formatting
with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    # Write sheets
    master_df.to_excel(writer, sheet_name='Product Fwd curve', index=False)
    eurdol_fwd_data.to_excel(writer, sheet_name='Euro-dollar Fwds', index=False)

    # Access workbook and worksheets
    workbook = writer.book
    sheet1 = writer.sheets['Product Fwd curve']
    sheet2 = writer.sheets['Euro-dollar Fwds']

    # Define formats
    center_format_2dp = workbook.add_format({
        'align': 'center', 'valign': 'vcenter', 'num_format': '0.00'
    })
    center_format_4dp = workbook.add_format({
        'align': 'center', 'valign': 'vcenter', 'num_format': '0.0000'
    })

    # Format and auto-size Product Fwd curve sheet
    for idx, col in enumerate(master_df.columns):
        # Determine max content width in this column
        max_len = max(
            master_df[col].astype(str).map(len).max(),
            len(str(col))
        ) + 2  # add padding

        # Choose format
        cell_format = center_format_4dp if col in rbob_cols_4dp else center_format_2dp

        # Set column width and format
        sheet1.set_column(idx, idx, max_len, cell_format)

    # Format and auto-size Euro-dollar Fwds sheet (all columns 4dp)
    for idx, col in enumerate(eurdol_fwd_data.columns):
        max_len = max(
            eurdol_fwd_data[col].astype(str).map(len).max(),
            len(str(col))
        ) + 2
        sheet2.set_column(idx, idx, max_len, center_format_4dp)

    # Freeze panes
    sheet1.freeze_panes(0, 1)  # Freeze first column
    sheet2.freeze_panes(1, 0)  # Freeze first row

print(f"Exported to: {filename}")

Exported to: cross_bbl_pricing_sheet_2025-08-06.xlsx


**Spend time cleaning up the script**

* Ensure we have the requirements list for the libraries compiled (ensure) no repeats
* Remove the CME data pulls for kero and sing-180
    * Maybe keep in the function to pull CME data but comment out
* Irrelevant bits/pieces of code

* Annotating functions/processes in detail:
    * `parse_barchart_symbols` : used to create the symbols for the respetive tickers - this becomes a list we can plug into the subsequent function to scrape price data  
    * `fetch_barchart_prices` : we take symbols created from the `symbols` list plug it as an input to our function to access the respective webpages and scrape the curve data into a df 
    * `clean_and_format_df` : initial clean of raw data - becomes easier to merge the curves later

* group the script by the flat price pulls, followed by how we derive the implied brent swap 
* Then the TS Column computations, followed by the cracks, blends for certain curves, then geographic arbs 
* Column re-organisation/re-labelling 
* Then the respective rounding that needs to be done for each curve

* Euro-dollar fwd curve data pull - into a neat dataframe
* Shrinking the Month, Year columns into a short & neat tenor - better readability 

* Then use the ExcelWriter to plug the assembled dfs into two separate sheets - centralising format, ensuring that the layout has been done well.
